In [ ]:
from langchain_core.runnables import (
    RunnableBranch,
    RunnableLambda,
    RunnableParallel,
    RunnablePassthrough,
)
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts.prompt import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from typing import Tuple, List, Optional
from langchain_core.messages import AIMessage, HumanMessage
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.document_loaders import WikipediaLoader
from langchain.text_splitter import TokenTextSplitter
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
from langchain_core.runnables import ConfigurableField, RunnableParallel, RunnablePassthrough

from Graph_RAG.base import *

/media/00.Data/00.User/02.wfx/app/anaconda3/envs/RAG/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)
/home/wfx/Code/Graph_RAG/base.py:9: LangChainDeprecationWarning: The class `Neo4jGraph` was deprecated in LangChain 0.3.8 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-neo4j package and should be used instead. To use it run `pip instal

In [2]:
os.getcwd()

'/home/wfx/Code'

In [3]:
import dotenv
dotenv.load_dotenv()

True

In [4]:
graph = Neo4jGraph(refresh_schema=False)

In [5]:
from langchain_text_splitters import TokenTextSplitter
from langchain_community.document_loaders import WikipediaLoader

# 读取维基百科文章
# raw_documents = WikipediaLoader(query="金庸", load_max_docs=5, lang='zh').load()
# raw_documents = WikipediaLoader(query="Elizabeth I").load()

In [6]:
# len(raw_documents)

In [7]:
# 定义分块策略
text_splitter = TokenTextSplitter(chunk_size=512, chunk_overlap=24)
# documents = text_splitter.split_documents(raw_documents[:3])

In [8]:
# import spacy
# from langchain.text_splitter import SpacyTextSplitter
# nlp = spacy.load("zh_core_web_sm")
# text_splitter = SpacyTextSplitter(
#     pipeline=nlp,
#     chunk_size = 512,
#     chunk_overlap  = 24
# )

In [9]:
# documents[0]

In [10]:
from qianfan.resources.console.iam import IAM
import os
os.environ["QIANFAN_ACCESS_KEY"] = "ALTAKL4iMnAPX8XNvNgebDnwnq"
os.environ["QIANFAN_SECRET_KEY"] = "02deb1efca3f4f358207ecfa7a8445d5"

# BEARER_TOKEN = IAM.create_bearer_token(expire_in_seconds=86400).body.get('token')
BEARER_TOKEN = ''

In [11]:
BEARER_TOKEN

''

In [12]:
from ERNIE import llm
# llm = LLM_for_Openai(BEARER_TOKEN)

In [13]:
llm('你好')

'你好！我是中文界面版的AI助手。可以用英语或中文和我聊天了。要是什么问题或者话题你想与我讨论，请随时问问看！'

In [14]:
llm.model_name

'llama3.1:latest'

In [15]:
from langchain_community.chat_models import ErnieBotChat
from langchain_experimental.graph_transformers import LLMGraphTransformer
llm_transformer = LLMGraphTransformer(llm=llm, ignore_tool_usage=True)

In [16]:
llm.model_name

'llama3.1:latest'

In [20]:
from langchain_core.documents import Document

In [21]:
Document('你好')

Document(metadata={}, page_content='你好')

In [22]:
with open('Graph_RAG/莲香.txt', 'r', encoding='utf-8') as f:
    text = f.read()
doc = Document(text)
doc

Document(metadata={}, page_content='孙太白尝言，其曾祖肄业于南山柳沟寺。麦秋旋里，经旬始返。启斋门，则案上尘生，窗间丝满，命仆粪除，至晚始觉清爽可坐。乃拂榻陈卧具，扁扉就枕，月色已满窗矣。辗转移时，万簌俱寂。忽闻风声隆隆，山门豁然作响，窃谓寺僧失扃。注念间，风声渐近居庐，俄而房门辟矣。大疑之，思未定，声已入屋。又有靴声铿铿然，渐傍寝门。心始怖。俄而寝门辟矣。忽视之，一大鬼鞠躬塞入，突立榻前，殆与梁齐。面似老瓜皮色，目光睒闪，绕室四顾，张巨口如盆，齿疏疏长三寸许，舌动喉鸣，呵喇之声，响连四壁，公惧极。又念咫尺之地势无所逃，不如因而刺之。乃阴抽枕下佩刀，遽拔而所之，中腹，作石缶声。鬼大怒，伸巨爪攫公。公少缩。鬼攫得衾捽之，忿忿而去。公随衾堕，伏地号呼。\n家人持火奔集，则门闭如故，排窗入，见公状，大骇。扶曳登床，始言其故。其验之，则衾夹于寝门之隙。启扉检照，见有爪痕如箕，五指着处皆穿。\n既明，不敢复留，负笈而归。后问僧人，无复他异。')

In [23]:
docs = text_splitter.split_documents([doc])

In [24]:
len(docs)

2

In [25]:
docs[0]

Document(metadata={}, page_content='孙太白尝言，其曾祖肄业于南山柳沟寺。麦秋旋里，经旬始返。启斋门，则案上尘生，窗间丝满，命仆粪除，至晚始觉清爽可坐。乃拂榻陈卧具，扁扉就枕，月色已满窗矣。辗转移时，万簌俱寂。忽闻风声隆隆，山门豁然作响，窃谓寺僧失扃。注念间，风声渐近居庐，俄而房门辟矣。大疑之，思未定，声已入屋。又有靴声铿铿然，渐傍寝门。心始怖。俄而寝门辟矣。忽视之，一大鬼鞠躬塞入，突立榻前，殆与梁齐。面似老瓜皮色，目光睒闪，绕室四顾，张巨口如盆，齿疏疏长三寸许，舌动喉鸣�')

In [26]:
graph_docs = []

In [27]:
# for doc_graph in llm_transformer.convert_to_graph_documents(docs):
#     graph_docs.append(doc_graph)

In [28]:
def Run(doc__):
    global graph_docs
    # print(doc__)
    res = llm_transformer.process_response(doc__)
    with threading.Lock():
        graph_docs.append(res)

In [29]:
import threading
from tqdm.notebook import tqdm, trange

In [30]:
docs[0]

Document(metadata={}, page_content='孙太白尝言，其曾祖肄业于南山柳沟寺。麦秋旋里，经旬始返。启斋门，则案上尘生，窗间丝满，命仆粪除，至晚始觉清爽可坐。乃拂榻陈卧具，扁扉就枕，月色已满窗矣。辗转移时，万簌俱寂。忽闻风声隆隆，山门豁然作响，窃谓寺僧失扃。注念间，风声渐近居庐，俄而房门辟矣。大疑之，思未定，声已入屋。又有靴声铿铿然，渐傍寝门。心始怖。俄而寝门辟矣。忽视之，一大鬼鞠躬塞入，突立榻前，殆与梁齐。面似老瓜皮色，目光睒闪，绕室四顾，张巨口如盆，齿疏疏长三寸许，舌动喉鸣�')

In [31]:
len(docs)

2

In [32]:
thread_num = 20

for star_idx in trange(0, len(docs), thread_num, desc='Running', unit='Epoch'):
    threads = []
    for i in range(thread_num):
        if star_idx + i >= len(docs):
            break
        # print(type(docs[star_idx + i]))
        t = threading.Thread(target=Run, args=[docs[star_idx + i]])
        threads.append(t)
        t.start()
    
    for t in threads:
        t.join()

Running:   0%|          | 0/1 [00:00<?, ?Epoch/s]

In [33]:
len(graph_docs)

2

In [34]:
if len(graph_docs) < 20:
    print(graph_docs)

[GraphDocument(nodes=[Node(id='衾', type='Entity', properties={}), Node(id='ghost', type='Entity', properties={}), Node(id='鬼', type='Entity', properties={}), Node(id='公', type='Person', properties={})], relationships=[Relationship(source=Node(id='公', type='Person', properties={}), target=Node(id='ghost', type='Entity', properties={}), type='WAS_ATTACKED_BY', properties={}), Relationship(source=Node(id='鬼', type='Entity', properties={}), target=Node(id='公', type='Person', properties={}), type='ATTACKED', properties={}), Relationship(source=Node(id='衾', type='Entity', properties={}), target=Node(id='鬼', type='Entity', properties={}), type='WAS_CLUTCHED_BY', properties={}), Relationship(source=Node(id='公', type='Person', properties={}), target=Node(id='鬼', type='Entity', properties={}), type='SUFFERED_CLAWING', properties={})], source=Document(metadata={}, page_content='�长三寸许，舌动喉鸣，呵喇之声，响连四壁，公惧极。又念咫尺之地势无所逃，不如因而刺之。乃阴抽枕下佩刀，遽拔而所之，中腹，作石缶声。鬼大怒，伸巨爪攫公。公少缩。鬼攫得衾捽之，忿忿而去。公随衾堕，伏地号呼。\n家人持火奔集，则门闭如故，排窗入，

In [35]:
len(set([x.source.page_content for x in graph_docs]))

2

In [36]:
graph_docs[0].source.page_content

'�长三寸许，舌动喉鸣，呵喇之声，响连四壁，公惧极。又念咫尺之地势无所逃，不如因而刺之。乃阴抽枕下佩刀，遽拔而所之，中腹，作石缶声。鬼大怒，伸巨爪攫公。公少缩。鬼攫得衾捽之，忿忿而去。公随衾堕，伏地号呼。\n家人持火奔集，则门闭如故，排窗入，见公状，大骇。扶曳登床，始言其故。其验之，则衾夹于寝门之隙。启扉检照，见有爪痕如箕，五指着处皆穿。\n既明，不敢复留，负笈而归。后问僧人，无复他异。'

In [37]:
for x in tqdm(graph_docs, desc='Up', unit='cnt'):
    try:
        graph.add_graph_documents(
          [x], 
          baseEntityLabel=True, 
          include_source=True
        )
    except Exception as e:
        print('ERROR: ', str(e))
        print(x)

Up:   0%|          | 0/2 [00:00<?, ?cnt/s]

In [40]:
# graph.add_graph_documents(
#   graph_docs[:-1], 
#   baseEntityLabel=True, 
#   include_source=True
# )

In [41]:
# from tqdm.notebook import tqdm
# for doc_ in tqdm(docs):
#     try:
#         graph_docs = llm_transformer.convert_to_graph_documents([doc_])
#     except Exception as e:
#         print('ERROR: ', str(e))
#         continue
#     graph.add_graph_documents(
#       graph_docs, 
#       baseEntityLabel=True, 
#       include_source=True
#     )

-
---
-

In [42]:
from neo4j import GraphDatabase

# 直接显示给定 Cypher 查询生成的图形
default_cypher = "MATCH (s)-[r:!MENTIONS]->(t) RETURN s,r,t LIMIT 50"

def showGraph(cypher: str = default_cypher):
    # create a neo4j session to run queries
    driver = GraphDatabase.driver(
        uri = os.environ["NEO4J_URI"],
        auth = (os.environ["NEO4J_USERNAME"],
                os.environ["NEO4J_PASSWORD"]))
    session = driver.session()
    widget = GraphWidget(graph = session.run(cypher).graph())
    widget.node_label_mapping = 'id'
    #display(widget)
    return widget

widget = showGraph()
widget

GraphWidget(layout=Layout(height='650px', width='100%'))

In [43]:
model_name = "sentence-transformers/all-MiniLM-L6-v2"
model_kwargs = {'device': 'cuda'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

In [41]:
vector_index = Neo4jVector.from_existing_graph(
    hf,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

In [50]:
# Retriever

graph.query(
    "CREATE FULLTEXT INDEX entity IF NOT EXISTS FOR (e:__Entity__) ON EACH [e.id]")

# Extract entities from text
class Entities(BaseModel):
    """Identifying information about entities.""" 

    names: List[str] = Field(
        ...,
        description="All the person, organization, or business entities that "
        "appear in the text",
    )

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "您正在从文本中提取 organization 和 person 实体。并且只返回json格式的输出！json中，只需要有res键，值为一个列表，内容你找到的所有organization和person实体名。",
        ),
        (
            "human",
            "使用给定的格式从以下内容中提取信息"
            "input: {question}",
        ),
    ]
)

import json
def res2json(res):
    try:
        resdic = json.loads(res)
    except:
        # pattern = r'```json(.*?)```'
        staridx = res.find('```json')
        resdic = res[staridx+len('```json'):res.find('```', staridx+1)]
        resdic = json.loads(resdic)
        # print(resdic)
        # print(type(resdic))
    return resdic
class outHelper:
    def __init__(self):
        pass
    def invoke(self, res) -> Entities:
        return res2json(res.content)
    
    def __call__(self, res) -> Entities:
        return res2json(res.content)

entity_chain = prompt | llm | outHelper()

In [51]:
entity_chain

ChatPromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='您正在从文本中提取 organization 和 person 实体。并且只返回json格式的输出！json中，只需要有res键，值为一个列表，内容你找到的所有organization和person实体名。'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='使用给定的格式从以下内容中提取信息input: {question}'), additional_kwargs={})])
| LLM_for_Openai(client=<openai.resources.chat.completions.Completions object at 0x71d3e743e4e0>, async_client=<openai.resources.chat.completions.AsyncCompletions object at 0x71d3e743c230>, root_client=<openai.OpenAI object at 0x71d3e79b7020>, root_async_client=<openai.AsyncOpenAI object at 0x71d3e7426060>, model_name='ernie-speed-128k', model_kwargs={}, openai_api_key=SecretStr('**********'))
| RunnableLambda(...)

In [52]:
req = '孙太白？'

In [53]:
x = prompt.invoke({"question": req})
x

ChatPromptValue(messages=[SystemMessage(content='您正在从文本中提取 organization 和 person 实体。并且只返回json格式的输出！json中，只需要有res键，值为一个列表，内容你找到的所有organization和person实体名。', additional_kwargs={}, response_metadata={}), HumanMessage(content='使用给定的格式从以下内容中提取信息input: 孙太白？', additional_kwargs={}, response_metadata={})])

In [54]:
res = llm.invoke(x)
res

AIMessage(content='```json\n{\n  "res": [\n    "孙太白（person）"\n  ]\n}\n```', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 26, 'prompt_tokens': 52, 'total_tokens': 78, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_name': 'ernie-speed-128k', 'system_fingerprint': None, 'finish_reason': 'normal', 'logprobs': None}, id='run-70808aee-c4aa-474e-b103-10d5b2b335ca-0', usage_metadata={'input_tokens': 52, 'output_tokens': 26, 'total_tokens': 78, 'input_token_details': {}, 'output_token_details': {}})

In [55]:
len(res2json(res.content).keys())

1

In [57]:
res = entity_chain.invoke({"question": req})
res['res']

['孙太白', 'person']

In [84]:
def generate_full_text_query(input: str) -> str:
    """为给定的输入字符串生成全文搜索查询。

    此函数构造适合全文搜索的查询字符串。
    它通过将输入字符串拆分为单词并附加一个
    相似度阈值（~2 个更改的字符）与每个单词匹配，然后组合
    他们使用 AND 运算符。用于映射用户问题中的实体
    添加到数据库值中，并允许一些拼写错误。"""
    full_text_query = ""
    words = [el for el in remove_lucene_chars(input).split() if el]
    for word in words[:-1]:
        full_text_query += f" {word}~2 AND"
    full_text_query += f" {words[-1]}~2"
    # return full_text_query.strip()
    return input

In [91]:
def structured_retriever(question: str, retry= 3, debug= False) -> str:
    """
    Collects the neighborhood of entities mentioned
    in the question
    """
    result = ""
    for i in range(retry): 
        try:
            entities = entity_chain.invoke({"question": question})
        
            if isinstance(entities['res'], list):
                break
        except Exception as e:
            print('ERROR: 格式错误: ', str(e))
    else:
        print('ERROR: 多次重试仍然《提取到的实体》格式错误')
        return ''
    for entity in entities['res']:
        if debug:
            print('分割出的实体列表为: ', str(entity))
        response = graph.query(
            """
            CALL db.index.fulltext.queryNodes('entity', $query, {limit:2})
            YIELD node,score
            MATCH (node)-[r]-(neighbor) // 通过这步匹配来明确neighbor的来源，可根据实际关系调整这里的模式
            CALL (node, neighbor) {
              MATCH (node)-[r:!MENTIONS]->(neighbor)
              RETURN node.id + ' - ' + type(r) + ' -> ' + neighbor.id AS output
              UNION ALL
              MATCH (node)<-[r:!MENTIONS]-(neighbor)
              RETURN neighbor.id + ' - ' + type(r) + ' -> ' +  node.id AS output
            }
            RETURN output LIMIT 50
            """,
            {"query": generate_full_text_query(entity)},
        )
        if debug:
            print('generate_full_text_query(entity): ', generate_full_text_query(entity))
        result += "\n".join([el['output'] for el in response])
    return result
    

In [92]:
structured_retriever(req, debug=True)

分割出的实体列表为:  孙太白
generate_full_text_query(entity):  孙太白
分割出的实体列表为:  person
generate_full_text_query(entity):  person


'孙太白 - 曾提及 -> 曾祖肄业于南山柳沟寺\n孙太白 - SPEAKS_OF -> 曾祖\n风声 - HEARD_BY -> 孙太白\n孙太白的曾祖 - 曾肄业于 -> 南山柳沟寺'

In [93]:
def retriever(question: str, debug=False, retry= 3):
    if debug:
        print(f"Search query: {question}")
    structured_data = structured_retriever(question, debug=debug, retry= retry)
    unstructured_data = [el.page_content for el in vector_index.similarity_search(question)]
    final_data = f"""Structured data:
{structured_data}
Unstructured data:
{"#Document ". join(unstructured_data)}
    """
    if debug:
        print(f'~ final_data({question}): ', final_data)
    return final_data

In [94]:
retriever(req, debug=True)

Search query: 孙太白？
分割出的实体列表为:  孙太白（person）
generate_full_text_query(entity):  孙太白（person）


Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

~ final_data(孙太白？):  Structured data:
孙太白 - 曾提及 -> 曾祖肄业于南山柳沟寺
孙太白 - SPEAKS_OF -> 曾祖
风声 - HEARD_BY -> 孙太白
孙太白的曾祖 - 曾肄业于 -> 南山柳沟寺
Unstructured data:

text: 孙太白尝言，其曾祖肄业于南山柳沟寺。麦秋旋里，经旬始返。启斋门，则案上尘生，窗间丝满，命仆粪除，至晚始觉清爽可坐。乃拂榻陈卧具，扁扉就枕，月色已满窗矣。辗转移时，万簌俱寂。忽闻风声隆隆，山门豁然作响，窃谓寺僧失扃。注念间，风声渐近居庐，俄而房门辟矣。大疑之，思未定，声已入屋。又有靴声铿铿然，渐傍寝门。心始怖。俄而寝门辟矣。忽视之，一大鬼鞠躬塞入，突立榻前，殆与梁齐。面似老瓜皮色，目光睒闪，绕室四顾，张巨口如盆，齿疏疏长三寸许，舌动喉鸣�#Document 
text: �长三寸许，舌动喉鸣，呵喇之声，响连四壁，公惧极。又念咫尺之地势无所逃，不如因而刺之。乃阴抽枕下佩刀，遽拔而所之，中腹，作石缶声。鬼大怒，伸巨爪攫公。公少缩。鬼攫得衾捽之，忿忿而去。公随衾堕，伏地号呼。
家人持火奔集，则门闭如故，排窗入，见公状，大骇。扶曳登床，始言其故。其验之，则衾夹于寝门之隙。启扉检照，见有爪痕如箕，五指着处皆穿。
既明，不敢复留，负笈而归。后问僧人，无复他异。
    


'Structured data:\n孙太白 - 曾提及 -> 曾祖肄业于南山柳沟寺\n孙太白 - SPEAKS_OF -> 曾祖\n风声 - HEARD_BY -> 孙太白\n孙太白的曾祖 - 曾肄业于 -> 南山柳沟寺\nUnstructured data:\n\ntext: 孙太白尝言，其曾祖肄业于南山柳沟寺。麦秋旋里，经旬始返。启斋门，则案上尘生，窗间丝满，命仆粪除，至晚始觉清爽可坐。乃拂榻陈卧具，扁扉就枕，月色已满窗矣。辗转移时，万簌俱寂。忽闻风声隆隆，山门豁然作响，窃谓寺僧失扃。注念间，风声渐近居庐，俄而房门辟矣。大疑之，思未定，声已入屋。又有靴声铿铿然，渐傍寝门。心始怖。俄而寝门辟矣。忽视之，一大鬼鞠躬塞入，突立榻前，殆与梁齐。面似老瓜皮色，目光睒闪，绕室四顾，张巨口如盆，齿疏疏长三寸许，舌动喉鸣�#Document \ntext: �长三寸许，舌动喉鸣，呵喇之声，响连四壁，公惧极。又念咫尺之地势无所逃，不如因而刺之。乃阴抽枕下佩刀，遽拔而所之，中腹，作石缶声。鬼大怒，伸巨爪攫公。公少缩。鬼攫得衾捽之，忿忿而去。公随衾堕，伏地号呼。\n家人持火奔集，则门闭如故，排窗入，见公状，大骇。扶曳登床，始言其故。其验之，则衾夹于寝门之隙。启扉检照，见有爪痕如箕，五指着处皆穿。\n既明，不敢复留，负笈而归。后问僧人，无复他异。\n    '

In [108]:
# Condense a chat history and follow-up question into a standalone question
_template = """Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question,
in its original language.
Chat History:
{chat_history}
Follow Up Input: {question}
Standalone question:"""  # noqa: E501
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

def _format_chat_history(chat_history: List[Tuple[str, str]]) -> List:
    buffer = []
    for human, ai in chat_history:
        buffer.append(HumanMessage(content=human))
        buffer.append(AIMessage(content=ai))
    return buffer

_search_query = RunnableBranch(
    # If input includes chat_history, we condense it with the follow-up question
    (
        RunnableLambda(lambda x: bool(x.get("chat_history"))).with_config(
            run_name="HasChatHistoryCheck"
        ),  # Condense follow-up question and chat into a standalone_question
        RunnablePassthrough.assign(
            chat_history=lambda x: _format_chat_history(x["chat_history"])
        )
        | CONDENSE_QUESTION_PROMPT
        | ChatOpenAI(temperature=0)
        | StrOutputParser(),
    ),
    # Else, we have no chat history, so just pass through the question
    RunnableLambda(lambda x : x["question"]),
)

In [130]:
template = """Answer the question based only on the following context:
{context}

Question: {question}
Use natural language and be concise.
Answer:"""
prompt = ChatPromptTemplate.from_template(template)

chain = (
    RunnableParallel(
        {
            "context": _search_query | retriever,
            "question": RunnablePassthrough(),
        }
    )
    | prompt
    # | llm
    # | StrOutputParser()
)

In [131]:
req = '鬼的描述'

In [132]:
res = chain.invoke({"question": req})

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: CALL subquery without a variable scope clause is now deprecated. Use CALL () { ... }} {position: line: 1, column: 1, offset: 0} for query: "CALL { CALL db.index.vector.queryNodes($index, $k, $embedding) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score UNION CALL db.index.fulltext.queryNodes($keyword_index, $query, {limit: $k}) YIELD node, score WITH collect({node:node, score:score}) AS nodes, max(score) AS max UNWIND nodes AS n RETURN n.node AS node, (n.score / max) AS score } WITH node, max(score) AS score ORDER BY score DESC LIMIT $k RETURN reduce(str='', k IN ['text'] | str + '\\n' + k + ': ' + coalesce(node[k], '')) AS text, node {.*, `embedding

In [133]:
res.messages[0].content

"Answer the question based only on the following context:\nStructured data:\n\nUnstructured data:\n\ntext: 孙太白尝言，其曾祖肄业于南山柳沟寺。麦秋旋里，经旬始返。启斋门，则案上尘生，窗间丝满，命仆粪除，至晚始觉清爽可坐。乃拂榻陈卧具，扁扉就枕，月色已满窗矣。辗转移时，万簌俱寂。忽闻风声隆隆，山门豁然作响，窃谓寺僧失扃。注念间，风声渐近居庐，俄而房门辟矣。大疑之，思未定，声已入屋。又有靴声铿铿然，渐傍寝门。心始怖。俄而寝门辟矣。忽视之，一大鬼鞠躬塞入，突立榻前，殆与梁齐。面似老瓜皮色，目光睒闪，绕室四顾，张巨口如盆，齿疏疏长三寸许，舌动喉鸣�#Document \ntext: �长三寸许，舌动喉鸣，呵喇之声，响连四壁，公惧极。又念咫尺之地势无所逃，不如因而刺之。乃阴抽枕下佩刀，遽拔而所之，中腹，作石缶声。鬼大怒，伸巨爪攫公。公少缩。鬼攫得衾捽之，忿忿而去。公随衾堕，伏地号呼。\n家人持火奔集，则门闭如故，排窗入，见公状，大骇。扶曳登床，始言其故。其验之，则衾夹于寝门之隙。启扉检照，见有爪痕如箕，五指着处皆穿。\n既明，不敢复留，负笈而归。后问僧人，无复他异。\n    \n\nQuestion: {'question': '鬼的描述'}\nUse natural language and be concise.\nAnswer:"

AuthenticationError: Error code: 401 - {'error': {'message': 'Incorrect API key provided: bce-v3/A***************************************************************57da. You can find your API key at https://platform.openai.com/account/api-keys.', 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}